# <center> Mean-Shift: Single Object Tracking in Images <center>


Object tracking is one of the most popular areas of video processing. The main purpose of 
object tracking is to estimate the position of the object in images continuously and reliably 
against dynamic scenes. This can be achieved by using the mean shift object tracking 
algorithm. The mean shift algorithm is an efficient approach to tracking objects whose 
appearance is defined by histograms. Thus, it can be used to track non-rigid objects by 
discovering clusters in a smooth density of samples.

### Question:
Using OpenCV implement a single object tracker. Steps to be implemented:
a) Use a pre-recorded video or your webcam to have a video Capture object.
b) Mark the region of interest (ROI or the object you want to track) using it
coordinates in the first frame.
c) Calculate the histogram of the ROI.
d) Iteratively calculate the histogram at each location (using cv2,calcBackProject)
and then apply mean shift to get the updated location of the ROI

### Procedure
 - Import Libraries
- Initialize the video captre from either a pre-recorded video or web cam
- Specific video path (Live – streaming path, recorded – log path)
- Creating ROI (Region of Interest)
- Number plate (ROI) – Value
- Create a condition to check whether that input of ROI is visible in the screen which being captured or not. 
- Calculate the histogram of ROI 
- Perform normalization with respect to CV2
- We need to create a frame start tracking loop to the frame end tracking .
- When the object is about to start, tracking loop must start
- If the object reached the destination, the tracking loop must stop (Terminate)
- Uber (Ride sharing – GPS vehicle. Tracking loop starts from source, terminates when object reaches destination)


In [1]:
# Import Libraries
import cv2
import numpy as np

# You can replace 'video_file_path' with the path to your pre-recorded video
# or set it to 0 to use your webcam
video_capture = cv2.VideoCapture(1)

ret, frame = video_capture.read()
if not ret:
    print("Failed to read the video frame.")
    exit()

x, y, w, h = cv2.selectROI("Select ROI", frame, fromCenter=False, showCrosshair=True)
roi = frame[y:y+h, x:x+w]

hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
hist_roi = cv2.calcHist([hsv_roi], [0], None, [180], [0, 180])
cv2.normalize(hist_roi, hist_roi, 0, 255, cv2.NORM_MINMAX)

term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

while True:
    ret, frame = video_capture.read()
    if not ret:
        break

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    back_project = cv2.calcBackProject([hsv], [0], hist_roi, [0, 180], 1)
    
    ret, track_window = cv2.meanShift(back_project, (x, y, w, h), term_crit)
    
    x, y, w, h = track_window
    
    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    cv2.imshow('Tracking', frame)
    
    if cv2.waitKey(30) & 0xFF == 27:
        break

video_capture.release()
cv2.destroyAllWindows()